In [ ]:
# %% [markdown]
# # Government Contract Document Processing Pipeline
# 
# This notebook provides a complete zero-shot solution for extracting structured information from government contract forms using transformer models.
# 
# **Features:**
# - Zero-shot extraction (no training required)
# - CPU-optimized processing
# - Handles PDF and image files
# - Batch processing for large datasets
# - Results visualization and export
# 
# **Academic Use:** LayoutLMv3 license allows academic research usage

In [10]:
import datasets

AttributeError: module 'pyarrow.parquet' has no attribute 'ParquetWriter'

In [6]:

# %% [markdown]
# ## 1. Setup and Installation
# %%
# Install required packages (run once)
import sys
import subprocess

def install_packages():
    """Install required packages if not already installed"""
    packages = [
        "torch",
        "transformers>=4.35.0",
        "datasets",
        "Pillow",
        "pdf2image",
        "pandas",
        "numpy",
        "tqdm",
        "matplotlib",
        "seaborn"
    ]
    
    for package in packages:
        try:
            __import__(package.split(">=")[0].split("==")[0])
            print(f"✓ {package} already installed")
        except ImportError:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Uncomment to install packages
install_packages()


✓ torch already installed
✓ transformers>=4.35.0 already installed


AttributeError: module 'pyarrow.parquet' has no attribute 'ParquetWriter'

In [4]:
!pip install pyarrow -Y


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -Y


In [ ]:

# %% [markdown]
# ## 2. Import Libraries and Configuration

# %%
import os
import json
import re
import time
from pathlib import Path
from typing import List, Dict, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# Core libraries
import torch
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
from PIL import Image, ImageDraw, ImageFont
import pdf2image
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import cv2  # For checkbox detection

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

print("✓ All libraries imported successfully")

# %%
# Configuration
CONFIG = {
    "model_name": "microsoft/layoutlmv3-base",  # LayoutLMv3 for form understanding
    "processor_name": "microsoft/layoutlmv3-base",  # Matching processor
    "batch_size": 5,  # Smaller batches for LayoutLMv3 (more memory intensive)
    "max_pages_per_doc": 5,  # Limit pages to process per document
    "image_dpi": 200,  # PDF to image conversion quality
    "timeout": 300,  # 5 minutes max per document
    "save_raw_text": False,  # Include raw OCR text in results
    "cache_dir": "./model_cache",  # Model cache directory
    "use_checkbox_detection": True,  # Enable checkbox detection
    "confidence_threshold": 0.7,  # Minimum confidence for field extraction
}

# Create directories
os.makedirs(CONFIG["cache_dir"], exist_ok=True)
os.makedirs("./results", exist_ok=True)
os.makedirs("./sample_data", exist_ok=True)

print("✓ Configuration set")
print(f"Model: {CONFIG['model_name']}")
print(f"Batch size: {CONFIG['batch_size']}")

# %% [markdown]
# ## 3. Document Processing Class

# %%
class ContractProcessor:
    """LayoutLMv3-based government contract document processor with checkbox detection"""
    
    def __init__(self, model_name: str = None, device: str = "cpu"):
        """Initialize the processor with LayoutLMv3"""
        
        self.model_name = model_name or CONFIG["model_name"]
        self.processor_name = CONFIG["processor_name"]
        self.device = device
        self.processor = None
        self.model = None
        
        print(f"Initializing ContractProcessor with LayoutLMv3")
        print(f"Model: {self.model_name}")
        print(f"Device: {self.device}")
    
    def load_model(self):
        """Load LayoutLMv3 model and processor"""
        if self.model is not None:
            return  # Already loaded
            
        print("Loading LayoutLMv3 model... This may take a few minutes on first run.")
        
        try:
            # Set cache directory
            os.environ["TRANSFORMERS_CACHE"] = CONFIG["cache_dir"]
            
            # Load LayoutLMv3 processor and model
            self.processor = LayoutLMv3Processor.from_pretrained(
                self.processor_name,
                cache_dir=CONFIG["cache_dir"]
            )
            
            self.model = LayoutLMv3ForTokenClassification.from_pretrained(
                self.model_name,
                torch_dtype=torch.float32,  # Use float32 for CPU
                device_map=self.device,
                cache_dir=CONFIG["cache_dir"]
            )
            
            print("✓ LayoutLMv3 model loaded successfully")
            
        except Exception as e:
            print(f"❌ Error loading LayoutLMv3 model: {e}")
            print("💡 Note: LayoutLMv3 requires more memory than simpler OCR models")
            raise
    
    def detect_checkboxes(self, image: Image.Image) -> Dict[str, bool]:
        """Detect checked boxes in contract forms using computer vision"""
        
        # Convert PIL image to OpenCV format
        cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
        
        checkboxes = {
            'professional_services': False,
            'grant': False,
            'lease': False,
            'attorney': False,
            'mou': False,
            'qpa': False,
            'procured_services': False,
            'maintenance': False,
            'license_agreement': False,
            'amendment': False,
            'renewal': False,
            'other': False
        }
        
        try:
            # Define checkbox regions based on typical form layout
            # These coordinates may need adjustment based on your specific forms
            checkbox_regions = {
                'professional_services': (100, 400, 500, 450),  # Approximate region
                'grant': (100, 450, 500, 500),
                'lease': (100, 500, 500, 550),
                # Add more regions as needed
            }
            
            # Look for X marks, checkmarks, or filled boxes
            for checkbox_name, (x1, y1, x2, y2) in checkbox_regions.items():
                # Extract region of interest
                if x2 < gray.shape[1] and y2 < gray.shape[0]:
                    roi = gray[y1:y2, x1:x2]
                    
                    # Simple detection: look for dark pixels in checkbox area
                    # This is a basic approach - can be enhanced with template matching
                    if roi.size > 0:
                        dark_pixel_ratio = np.sum(roi < 128) / roi.size
                        if dark_pixel_ratio > 0.1:  # Threshold for "marked"
                            checkboxes[checkbox_name] = True
            
            # Alternative: Text-based detection for more reliability
            # Convert to text and look for patterns like "X Professional/Personal Services"
            
        except Exception as e:
            print(f"Checkbox detection error: {e}")
        
        return checkboxes
    
    def extract_with_layoutlm(self, image: Image.Image) -> Dict[str, Any]:
        """Extract structured information using LayoutLMv3"""
        
        try:
            # Process image with LayoutLMv3
            encoding = self.processor(image, return_tensors="pt")
            
            # Get model predictions
            with torch.no_grad():
                outputs = self.model(**encoding)
            
            # Extract predictions (this is a simplified approach)
            # In practice, you'd train LayoutLMv3 on your specific forms
            predictions = outputs.logits.argmax(-1)
            
            # For now, we'll use OCR text extraction from LayoutLMv3
            # and then apply our parsing logic
            text = self.processor.tokenizer.batch_decode(
                encoding.input_ids, skip_special_tokens=True
            )[0]
            
            return {'text': text, 'confidence': 0.8}
            
        except Exception as e:
            print(f"LayoutLMv3 extraction error: {e}")
            return {'text': '', 'confidence': 0.0}
    
    def extract_text_from_image(self, image: Image.Image) -> str:
        """Extract text from image using LayoutLMv3 with form understanding"""
        try:
            # Load model if not already loaded
            if self.model is None:
                self.load_model()
            
            # Use LayoutLMv3 for structured extraction
            result = self.extract_with_layoutlm(image)
            
            # Detect checkboxes
            if CONFIG["use_checkbox_detection"]:
                checkboxes = self.detect_checkboxes(image)
                
                # Add checkbox information to text
                checkbox_text = "\n=== CHECKBOX DETECTION ===\n"
                for checkbox_name, is_checked in checkboxes.items():
                    if is_checked:
                        checkbox_text += f"{checkbox_name}: CHECKED\n"
                
                result['text'] += checkbox_text
            
            return result['text']
            
        except Exception as e:
            print(f"LayoutLMv3 text extraction failed: {e}")
            return ""
    
    def parse_contract_fields(self, text: str) -> Dict[str, str]:
        """Parse contract fields using enhanced patterns with section awareness"""
        
        fields = {
            'eds_number': '',                    # 1. EDS Number
            'date_prepared': '',                 # 2. Date prepared
            'contracts_leases': '',              # 3. Contracts & Leases
            'account_number': '',                # 4. Account Number
            'account_name': '',                  # 5. Account Name
            'total_amount_this_action': '',      # 6. Total amount this action
            'new_contract_total': '',            # 7. New contract total
            'revenue_generated_this_action': '', # 8. Revenue generated this action
            'revenue_generated_total_contract': '', # 9. Revenue generated total contract
            'from_date': '',                     # 11. From (month, day, year)
            'to_date': '',                       # 12. To (month, day, year)
            'method_source_selection': '',       # 13. Method of source selection
            'email_address': '',                 # 19. E-mail address
            'vendor_id': '',                     # 23. Vendor ID #
            'vendor_name': '',                   # 24. Name
            'primary_vendor_mwbe': '',           # 29. Primary Vendor: M/WBE
            'sub_vendor_mwbe': '',               # 31. Sub Vendor:M/WBE
            'renewal_language': '',              # 33. Is there Renewal Language in the document?
            'termination_convenience_clause': '',# 34. Is there a "Termination for Convenience" clause
            'description_work_justification': '' # 37. Description of work and justification for spending money
        }
        
        # Extract sections for better context
        sections = self.extract_sections(text)
        
        # Enhanced regex patterns with section awareness
        patterns = {
            'eds_number': [
                r'1\.\s*EDS Number[:\s]*([^\n\r]+)',
                r'EDS Number[:\s]*([A-Z0-9\-]+)',
                r'([A-Z]\d{2}-\d+-\d{4})',
            ],
            'date_prepared': [
                r'2\.\s*Date prepared[:\s]*([^\n\r]+)',
                r'Date prepared[:\s]*([^\n\r]+)',
                r'(\d{1,2}\/\d{1,2}\/\d{4})',
            ],
            'contracts_leases': [
                r'3\.\s*CONTRACTS & LEASES[:\s]*([^\n\r]+)',
                r'Professional/Personal Services.*?CHECKED',
                r'Grant.*?CHECKED',
                r'Lease.*?CHECKED',
                # Will be enhanced by checkbox detection
            ],
            'account_number': [
                r'4\.\s*Account Number[:\s]*([^\n\r]+)',
                r'Account Number[:\s]*([^\n\r]+)',
                r'(\d{4}-?\d+)',
            ],
            'account_name': [
                r'5\.\s*Account Name[:\s]*([^\n\r]+)',
                r'Account Name[:\s]*([^\n\r]+)',
            ],
            'total_amount_this_action': [
                r'6\.\s*Total amount this action[:\s]*\$?([\d,]+\.?\d*)',
                r'Total amount this action[:\s]*\$?([\d,]+\.?\d*)',
                r'6\.\s*Total amount[:\s]*\$?([\d,]+\.?\d*)',
            ],
            'new_contract_total': [
                r'7\.\s*New contract total[:\s]*\$?([\d,]+\.?\d*)',
                r'New contract total[:\s]*\$?([\d,]+\.?\d*)',
            ],
            'revenue_generated_this_action': [
                r'8\.\s*Revenue generated this action[:\s]*\$?([\d,]+\.?\d*)',
                r'Revenue generated this action[:\s]*\$?([\d,]+\.?\d*)',
            ],
            'revenue_generated_total_contract': [
                r'9\.\s*Revenue generated total contract[:\s]*\$?([\d,]+\.?\d*)',
                r'Revenue generated total contract[:\s]*\$?([\d,]+\.?\d*)',
            ],
            'from_date': [
                r'11\.\s*From \(month, day, year\)[:\s]*([^\n\r]+)',
                r'From \(month, day, year\)[:\s]*([^\n\r]+)',
                r'11\.\s*From.*?(\d{1,2}\/\d{1,2}\/\d{4})',
                r'(\d{1,2}\/\d{1,2}\/\d{4})\s+\d{1,2}\/\d{1,2}\/\d{4}',  # First date in pair
            ],
            'to_date': [
                r'12\.\s*To \(month, day, year\)[:\s]*([^\n\r]+)',
                r'To \(month, day, year\)[:\s]*([^\n\r]+)',
                r'12\.\s*To.*?(\d{1,2}\/\d{1,2}\/\d{4})',
                r'\d{1,2}\/\d{1,2}\/\d{4}\s+(\d{1,2}\/\d{1,2}\/\d{4})',  # Second date in pair
            ],
            'method_source_selection': [
                r'13\.\s*Method of source selection[:\s]*([^\n\r]+)',
                r'Method of source selection[:\s]*([^\n\r]+)',
                r'(Bid/Quotation|RFP|Emergency|Negotiated|Special Procurement)',
            ],
            'email_address': [
                r'19\.\s*E-mail address[:\s]*([^\n\r]+)',
                r'E-mail address[:\s]*([^\n\r]+)',
                r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})',
            ],
            'vendor_id': [
                r'23\.\s*Vendor ID #[:\s]*([^\n\r]+)',
                r'Vendor ID #[:\s]*([^\n\r]+)',
                r'VENDOR INFORMATION.*?(\d{10})',
                r'(\d{10})',
            ],
            'vendor_name': [
                r'24\.\s*Name[:\s]*([^\n\r]+)',
                r'VENDOR INFORMATION.*?24\.\s*Name[:\s]*([^\n\r]+)',
                r'VENDOR INFORMATION.*?Name[:\s]*([^\n\r]+)',
            ],
            'primary_vendor_mwbe': [
                r'29\.\s*Primary Vendor: M/WBE[:\s]*([^\n\r]+)',
                r'Primary Vendor: M/WBE[:\s]*([^\n\r]+)',
                r'Primary.*?M/WBE.*?(Yes|No)',
            ],
            'sub_vendor_mwbe': [
                r'31\.\s*Sub Vendor:M/WBE[:\s]*([^\n\r]+)',
                r'Sub Vendor:M/WBE[:\s]*([^\n\r]+)',
                r'Sub.*?M/WBE.*?(Yes|No)',
            ],
            'renewal_language': [
                r'33\.\s*Is there Renewal Language in the document\?[:\s]*([^\n\r]+)',
                r'Is there Renewal Language in the document\?[:\s]*([^\n\r]+)',
                r'Renewal Language.*?(Yes|No)',
            ],
            'termination_convenience_clause': [
                r'34\.\s*Is there a "Termination for Convenience" clause in the document\?[:\s]*([^\n\r]+)',
                r'Is there a "Termination for Convenience" clause.*?(Yes|No)',
                r'Termination.*?Convenience.*?(Yes|No)',
            ],
            'description_work_justification': [
                r'37\.\s*Description of work and justification for spending money[:\s]*([^\n\r]{1,500})',
                r'Description of work and justification for spending money[:\s]*([^\n\r]{1,500})',
                r'Description.*?work.*?justification[:\s]*([^\n\r]{1,500})',
            ]
        }
        
        # Extract fields using patterns with section context
        for field_name, field_patterns in patterns.items():
            for pattern in field_patterns:
                # Try pattern on full text first
                match = re.search(pattern, text, re.IGNORECASE | re.MULTILINE | re.DOTALL)
                if match:
                    extracted_value = match.group(1).strip() if match.lastindex >= 1 else match.group(0).strip()
                    if extracted_value and len(extracted_value) < 500:  # Sanity check
                        fields[field_name] = extracted_value
                        break
                
                # If not found, try in relevant section
                relevant_section = self.get_relevant_section(field_name, sections)
                if relevant_section:
                    match = re.search(pattern, relevant_section, re.IGNORECASE | re.MULTILINE)
                    if match:
                        extracted_value = match.group(1).strip() if match.lastindex >= 1 else match.group(0).strip()
                        if extracted_value and len(extracted_value) < 500:
                            fields[field_name] = extracted_value
                            break
        
        # Post-process checkbox information
        fields = self.process_checkbox_info(text, fields)
        
        return fields
    
    def extract_sections(self, text: str) -> Dict[str, str]:
        """Extract different sections of the form for better context"""
        sections = {}
        
        section_patterns = {
            'agency_info': (r'AGENCY INFORMATION', r'COURIER INFORMATION'),
            'courier_info': (r'COURIER INFORMATION', r'VENDOR INFORMATION'),
            'vendor_info': (r'VENDOR INFORMATION', r'FISCAL INFORMATION'),
            'fiscal_info': (r'FISCAL INFORMATION', r'TIME PERIOD'),
            'time_period': (r'TIME PERIOD COVERED', r'Method of source selection'),
            'contracts_leases': (r'CONTRACTS & LEASES', r'FISCAL INFORMATION'),
        }
        
        for section_name, (start_pattern, end_pattern) in section_patterns.items():
            start_match = re.search(start_pattern, text, re.IGNORECASE)
            end_match = re.search(end_pattern, text, re.IGNORECASE)
            
            if start_match:
                start_pos = start_match.end()
                end_pos = end_match.start() if end_match else len(text)
                sections[section_name] = text[start_pos:end_pos]
        
        return sections
    
    def get_relevant_section(self, field_name: str, sections: Dict[str, str]) -> str:
        """Get the most relevant section for a field"""
        field_section_map = {
            'vendor_id': 'vendor_info',
            'vendor_name': 'vendor_info',
            'account_number': 'fiscal_info',
            'account_name': 'fiscal_info',
            'total_amount_this_action': 'fiscal_info',
            'new_contract_total': 'fiscal_info',
            'from_date': 'time_period',
            'to_date': 'time_period',
            'contracts_leases': 'contracts_leases',
        }
        
        section_name = field_section_map.get(field_name)
        return sections.get(section_name, '') if section_name else ''
    
    def process_checkbox_info(self, text: str, fields: Dict[str, str]) -> Dict[str, str]:
        """Process checkbox detection results"""
        
        # Look for checkbox indicators in text
        checkbox_indicators = [
            'professional_services_checked',
            'grant_checked', 
            'lease_checked',
            'other_checked'
        ]
        
        contract_types = []
        
        # Check for each type
        if re.search(r'Professional/Personal Services.*?CHECKED', text, re.IGNORECASE):
            contract_types.append('Professional/Personal Services')
        elif re.search(r'X.*?Professional/Personal Services|Professional/Personal Services.*?X', text, re.IGNORECASE):
            contract_types.append('Professional/Personal Services')
            
        if re.search(r'Grant.*?CHECKED', text, re.IGNORECASE):
            contract_types.append('Grant')
        elif re.search(r'X.*?Grant|Grant.*?X', text, re.IGNORECASE):
            contract_types.append('Grant')
            
        if re.search(r'Other.*?CHECKED', text, re.IGNORECASE):
            # Look for the "Other" specification
            other_match = re.search(r'Other[:\s]*([^\n\r]+)', text, re.IGNORECASE)
            if other_match:
                contract_types.append(f"Other: {other_match.group(1).strip()}")
        
        # Update contracts_leases field
        if contract_types:
            fields['contracts_leases'] = ', '.join(contract_types)
        
        return fields
        
        # Extract fields using patterns
        for field_name, field_patterns in patterns.items():
            for pattern in field_patterns:
                match = re.search(pattern, text, re.IGNORECASE | re.MULTILINE)
                if match:
                    extracted_value = match.group(1).strip()
                    if extracted_value and len(extracted_value) < 200:  # Sanity check
                        fields[field_name] = extracted_value
                        break
        
        return fields
    
    def process_document(self, file_path: str) -> Dict[str, Any]:
        """Process a single contract document"""
        
        start_time = time.time()
        
        result = {
            'file_path': file_path,
            'filename': os.path.basename(file_path),
            'status': 'processing',
            'processing_time': 0,
            'error': None,
            'pages_processed': 0,
            'extracted_fields': {},
            'raw_text': ''
        }
        
        try:
            # Load model if not already loaded
            if self.model is None:
                self.load_model()
            
            # Handle different file types
            if file_path.lower().endswith('.pdf'):
                images = self.pdf_to_images(file_path)
            else:
                # Assume image file
                images = [Image.open(file_path)]
            
            if not images:
                raise ValueError("No images extracted from document")
            
            # Process all pages
            all_text_parts = []
            for i, image in enumerate(images[:CONFIG["max_pages_per_doc"]]):
                page_text = self.extract_text_from_image(image)
                if page_text:
                    all_text_parts.append(f"=== Page {i+1} ===\n{page_text}")
            
            # Combine all text
            combined_text = "\n\n".join(all_text_parts)
            
            # Parse structured fields
            extracted_fields = self.parse_contract_fields(combined_text)
            
            # Update result
            result.update({
                'status': 'success',
                'extracted_fields': extracted_fields,
                'pages_processed': len(images),
                'processing_time': time.time() - start_time,
                'raw_text': combined_text if CONFIG["save_raw_text"] else ''
            })
            
        except Exception as e:
            result.update({
                'status': 'failed',
                'error': str(e),
                'processing_time': time.time() - start_time
            })
        
        return result
    
    def process_batch(self, file_paths: List[str]) -> List[Dict[str, Any]]:
        """Process a batch of documents with progress bar"""
        
        results = []
        
        with tqdm(total=len(file_paths), desc="Processing contracts") as pbar:
            for file_path in file_paths:
                result = self.process_document(file_path)
                results.append(result)
                
                # Update progress bar
                status_icon = "✓" if result['status'] == 'success' else "❌"
                pbar.set_postfix({
                    'file': os.path.basename(file_path)[:20],
                    'status': status_icon
                })
                pbar.update(1)
        
        return results

# %%
# Initialize the processor
processor = ContractProcessor()
print("✓ ContractProcessor initialized")

# %% [markdown]
# ## 4. Test on Sample Documents

# %%
# Test the processor on a single document
def test_single_document(file_path: str):
    """Test processing on a single document"""
    
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        print("\n💡 To test the processor:")
        print("1. Place a sample contract PDF in the './sample_data/' folder")
        print("2. Update the file_path variable below")
        print("3. Run this cell again")
        return None
    
    print(f"🔄 Testing on: {file_path}")
    print("=" * 50)
    
    result = processor.process_document(file_path)
    
    # Display results
    print(f"Status: {result['status']}")
    print(f"Processing time: {result['processing_time']:.2f} seconds")
    print(f"Pages processed: {result['pages_processed']}")
    
    if result['status'] == 'success':
        print("\n📋 Extracted Fields:")
        for field, value in result['extracted_fields'].items():
            if value:  # Only show non-empty fields
                print(f"  {field}: {value}")
        
        if not any(result['extracted_fields'].values()):
            print("  ⚠️ No fields extracted. Raw text preview:")
            print(f"  {result['raw_text'][:200]}...")
    else:
        print(f"\n❌ Error: {result['error']}")
    
    return result

# Test with a sample file (update path as needed)
sample_file = "./sample_data/sample_contract.pdf"

# Uncomment to test with your own file:
# test_result = test_single_document(sample_file)

# %% [markdown]
# ## 5. Batch Processing Function

# %%
def process_document_directory(input_dir: str, file_extensions: List[str] = None) -> pd.DataFrame:
    """Process all documents in a directory"""
    
    if file_extensions is None:
        file_extensions = ['.pdf', '.png', '.jpg', '.jpeg', '.tiff', '.tif']
    
    # Find all contract files
    file_paths = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if any(file.lower().endswith(ext) for ext in file_extensions):
                file_paths.append(os.path.join(root, file))
    
    if not file_paths:
        print(f"❌ No files found in {input_dir} with extensions {file_extensions}")
        return pd.DataFrame()
    
    print(f"📁 Found {len(file_paths)} files to process")
    print(f"📊 Processing in batches of {CONFIG['batch_size']}")
    
    # Process in batches
    all_results = []
    
    for i in range(0, len(file_paths), CONFIG['batch_size']):
        batch_files = file_paths[i:i + CONFIG['batch_size']]
        batch_num = i // CONFIG['batch_size'] + 1
        total_batches = (len(file_paths) + CONFIG['batch_size'] - 1) // CONFIG['batch_size']
        
        print(f"\n🔄 Processing batch {batch_num}/{total_batches}")
        
        batch_results = processor.process_batch(batch_files)
        all_results.extend(batch_results)
        
        # Show batch summary
        successful = sum(1 for r in batch_results if r['status'] == 'success')
        print(f"   ✓ {successful}/{len(batch_results)} successful")
    
    # Convert to DataFrame
    df = create_results_dataframe(all_results)
    
    # Summary statistics
    total_successful = (df['status'] == 'success').sum()
    success_rate = (total_successful / len(df)) * 100
    avg_time = df[df['status'] == 'success']['processing_time'].mean()
    
    print(f"\n📊 PROCESSING COMPLETE")
    print(f"   Total files: {len(df)}")
    print(f"   Successful: {total_successful}")
    print(f"   Success rate: {success_rate:.1f}%")
    print(f"   Average time: {avg_time:.2f}s per document")
    
    return df

def create_results_dataframe(results: List[Dict[str, Any]]) -> pd.DataFrame:
    """Convert results list to structured DataFrame"""
    
    records = []
    
    for result in results:
        # Base record
        record = {
            'filename': result['filename'],
            'file_path': result['file_path'],
            'status': result['status'],
            'processing_time': result['processing_time'],
            'pages_processed': result['pages_processed'],
            'error': result.get('error', '')
        }
        
        # Add extracted fields
        if result['status'] == 'success':
            record.update(result['extracted_fields'])
        
        records.append(record)
    
    return pd.DataFrame(records)

# %% [markdown]
# ## 6. Process Your Documents

# %%
# MAIN PROCESSING SECTION
# Update this path to point to your contract documents
INPUT_DIRECTORY = "./sample_data"

# Process documents (uncomment when ready)
print("🚀 Ready to process documents!")
print(f"Input directory: {INPUT_DIRECTORY}")
print(f"Configuration: {CONFIG}")

# Uncomment the following lines to start processing:
# df_results = process_document_directory(INPUT_DIRECTORY)

# For now, let's create some sample results for demonstration
print("\n💡 To process your documents:")
print("1. Place your contract files in a directory")
print("2. Update INPUT_DIRECTORY above")
print("3. Uncomment the processing line")
print("4. Run this cell")

# %% [markdown]
# ## 7. Results Analysis and Visualization

# %%
# Create sample data for demonstration (replace with actual results)
def create_sample_results():
    """Create sample results for demonstration purposes"""
    
    sample_data = [
        {
            'filename': 'contract_001.pdf',
            'status': 'success',
            'processing_time': 25.3,
            'pages_processed': 2,
            'eds_number': 'C44P-5-121',
            'total_amount_this_action': '150000.00',
            'new_contract_total': '150000.00',
            'vendor_name': 'ABC Corporation',
            'from_date': '01/01/2023',
            'to_date': '12/31/2023',
            'account_number': '6000-154600',
            'vendor_id': '0000054625'
        },
        {
            'filename': 'contract_002.pdf', 
            'status': 'success',
            'processing_time': 18.7,
            'pages_processed': 1,
            'eds_number': 'C45A-6-789',
            'total_amount_this_action': '75000.00',
            'vendor_name': 'XYZ Services',
            'from_date': '03/15/2023',
            'to_date': '03/14/2024',
            'email_address': 'contract@xyz.com'
        },
        {
            'filename': 'contract_003.pdf',
            'status': 'failed',
            'processing_time': 45.1,
            'pages_processed': 0,
            'error': 'PDF conversion failed'
        }
    ]
    
    return pd.DataFrame(sample_data)

# Use sample data for now (replace with df_results from actual processing)
df_results = create_sample_results()
print("📊 Sample results loaded for demonstration")

# %%
def analyze_results(df: pd.DataFrame):
    """Analyze and visualize processing results"""
    
    if df.empty:
        print("No results to analyze")
        return
    
    print("📈 RESULTS ANALYSIS")
    print("=" * 50)
    
    # Basic statistics
    total_docs = len(df)
    successful = (df['status'] == 'success').sum()
    failed = (df['status'] == 'failed').sum()
    success_rate = (successful / total_docs) * 100
    
    print(f"Total documents: {total_docs}")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Success rate: {success_rate:.1f}%")
    
    if successful > 0:
        successful_df = df[df['status'] == 'success']
        avg_time = successful_df['processing_time'].mean()
        avg_pages = successful_df['pages_processed'].mean()
        
        print(f"Average processing time: {avg_time:.2f}s")
        print(f"Average pages per document: {avg_pages:.1f}")
    
    # Field extraction rates
    print(f"\n📋 Field Extraction Rates:")
    field_columns = [
        'eds_number', 'date_prepared', 'account_number', 'account_name',
        'total_amount_this_action', 'new_contract_total', 'from_date', 'to_date',
        'vendor_id', 'vendor_name', 'email_address'
    ]
    
    for field in field_columns:
        if field in df.columns:
            non_empty = df[field].notna() & (df[field] != '')
            rate = (non_empty.sum() / successful) * 100 if successful > 0 else 0
            print(f"  {field}: {rate:.1f}%")
    
    # Visualizations
    create_visualizations(df)

def create_visualizations(df: pd.DataFrame):
    """Create visualizations of the results"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Contract Processing Results Analysis', fontsize=16)
    
    # 1. Success/Failure pie chart
    status_counts = df['status'].value_counts()
    axes[0, 0].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%')
    axes[0, 0].set_title('Processing Status Distribution')
    
    # 2. Processing time histogram
    successful_df = df[df['status'] == 'success']
    if not successful_df.empty:
        axes[0, 1].hist(successful_df['processing_time'], bins=10, alpha=0.7)
        axes[0, 1].set_xlabel('Processing Time (seconds)')
        axes[0, 1].set_ylabel('Number of Documents')
        axes[0, 1].set_title('Processing Time Distribution')
    else:
        axes[0, 1].text(0.5, 0.5, 'No successful\nprocessing times', 
                       ha='center', va='center', transform=axes[0, 1].transAxes)
        axes[0, 1].set_title('Processing Time Distribution')
    
    # 3. Field extraction success rates
    field_columns = [
        'eds_number', 'date_prepared', 'account_number', 'total_amount_this_action',
        'vendor_name', 'from_date', 'to_date', 'email_address'
    ]
    
    field_rates = []
    field_names = []
    
    for field in field_columns:
        if field in df.columns:
            non_empty = df[field].notna() & (df[field] != '')
            rate = (non_empty.sum() / len(successful_df)) * 100 if len(successful_df) > 0 else 0
            field_rates.append(rate)
            field_names.append(field.replace('_', ' ').title())
    
    if field_rates:
        bars = axes[1, 0].bar(field_names, field_rates)
        axes[1, 0].set_ylabel('Extraction Rate (%)')
        axes[1, 0].set_title('Field Extraction Success Rates')
        axes[1, 0].tick_params(axis='x', rotation=45)
        
        # Color bars based on success rate
        for bar, rate in zip(bars, field_rates):
            if rate >= 80:
                bar.set_color('green')
            elif rate >= 60:
                bar.set_color('orange')
            else:
                bar.set_color('red')
    
    # 4. Pages processed distribution
    if not successful_df.empty and 'pages_processed' in successful_df.columns:
        pages_counts = successful_df['pages_processed'].value_counts().sort_index()
        axes[1, 1].bar(pages_counts.index, pages_counts.values)
        axes[1, 1].set_xlabel('Number of Pages')
        axes[1, 1].set_ylabel('Number of Documents')
        axes[1, 1].set_title('Pages Processed Distribution')
    else:
        axes[1, 1].text(0.5, 0.5, 'No page count\ndata available', 
                       ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Pages Processed Distribution')
    
    plt.tight_layout()
    plt.show()

# Analyze the results
analyze_results(df_results)

# %% [markdown]
# ## 8. Export Results

# %%
def export_results(df: pd.DataFrame, output_dir: str = "./results"):
    """Export results to various formats"""
    
    if df.empty:
        print("No results to export")
        return
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Export to CSV
    csv_path = os.path.join(output_dir, "contract_extraction_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"✓ Results exported to CSV: {csv_path}")
    
    # Export to JSON
    json_path = os.path.join(output_dir, "contract_extraction_results.json")
    df.to_json(json_path, orient='records', indent=2)
    print(f"✓ Results exported to JSON: {json_path}")
    
    # Export to Excel with multiple sheets
    excel_path = os.path.join(output_dir, "contract_extraction_results.xlsx")
    with pd.ExcelWriter(excel_path) as writer:
        # All results
        df.to_excel(writer, sheet_name='All_Results', index=False)
        
        # Successful extractions only
        successful_df = df[df['status'] == 'success']
        if not successful_df.empty:
            successful_df.to_excel(writer, sheet_name='Successful_Extractions', index=False)
        
        # Failed extractions
        failed_df = df[df['status'] == 'failed']
        if not failed_df.empty:
            failed_df.to_excel(writer, sheet_name='Failed_Extractions', index=False)
        
        # Summary statistics
        summary_stats = create_summary_stats(df)
        summary_stats.to_excel(writer, sheet_name='Summary', index=True)
    
    print(f"✓ Results exported to Excel: {excel_path}")
    
    # Create processing report
    report_path = os.path.join(output_dir, "processing_report.txt")
    create_processing_report(df, report_path)
    print(f"✓ Processing report: {report_path}")

def create_summary_stats(df: pd.DataFrame) -> pd.DataFrame:
    """Create summary statistics DataFrame"""
    
    stats = {
        'Total Documents': len(df),
        'Successful Extractions': (df['status'] == 'success').sum(),
        'Failed Extractions': (df['status'] == 'failed').sum(),
        'Success Rate (%)': ((df['status'] == 'success').sum() / len(df)) * 100,
    }
    
    if (df['status'] == 'success').any():
        successful_df = df[df['status'] == 'success']
        stats.update({
            'Average Processing Time (s)': successful_df['processing_time'].mean(),
            'Total Processing Time (s)': successful_df['processing_time'].sum(),
            'Average Pages per Document': successful_df['pages_processed'].mean(),
        })
    
    return pd.DataFrame(list(stats.items()), columns=['Metric', 'Value'])

def create_processing_report(df: pd.DataFrame, output_path: str):
    """Create a detailed processing report"""
    
    with open(output_path, 'w') as f:
        f.write("GOVERNMENT CONTRACT PROCESSING REPORT\n")
        f.write("=" * 50 + "\n\n")
        
        # Basic statistics
        f.write("PROCESSING SUMMARY\n")
        f.write("-" * 20 + "\n")
        f.write(f"Total documents processed: {len(df)}\n")
        f.write(f"Successful extractions: {(df['status'] == 'success').sum()}\n")
        f.write(f"Failed extractions: {(df['status'] == 'failed').sum()}\n")
        f.write(f"Success rate: {((df['status'] == 'success').sum() / len(df)) * 100:.1f}%\n\n")
        
        # Field extraction rates
        f.write("FIELD EXTRACTION RATES\n")
        f.write("-" * 25 + "\n")
        
        field_columns = [
            'eds_number', 'date_prepared', 'account_number', 'account_name',
            'total_amount_this_action', 'new_contract_total', 'from_date', 'to_date',
            'vendor_id', 'vendor_name', 'email_address', 'method_source_selection'
        ]
        
        successful_count = (df['status'] == 'success').sum()
        
        for field in field_columns:
            if field in df.columns:
                non_empty = df[field].notna() & (df[field] != '')
                rate = (non_empty.sum() / successful_count) * 100 if successful_count > 0 else 0
                f.write(f"{field.replace('_', ' ').title()}: {rate:.1f}%\n")
        
        # Failed files
        failed_df = df[df['status'] == 'failed']
        if not failed_df.empty:
            f.write(f"\nFAILED EXTRACTIONS ({len(failed_df)} files)\n")
            f.write("-" * 30 + "\n")
            for _, row in failed_df.iterrows():
                f.write(f"File: {row['filename']}\n")
                f.write(f"Error: {row.get('error', 'Unknown error')}\n\n")

# Export results
export_results(df_results)

# %% [markdown]
# ## 9. Advanced Processing Options

# %%
def process_large_dataset(input_dir: str, checkpoint_interval: int = 100):
    """Process large datasets with checkpointing for recovery"""
    
    print("🚀 LARGE DATASET PROCESSING MODE")
    print("Features: Checkpointing, Progress saving, Error recovery")
    
    # Find all files
    file_paths = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.pdf', '.png', '.jpg', '.jpeg')):
                file_paths.append(os.path.join(root, file))
    
    print(f"📁 Found {len(file_paths)} files to process")
    
    # Check for existing checkpoint
    checkpoint_path = "./results/processing_checkpoint.json"
    processed_files = set()
    
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'r') as f:
            checkpoint_data = json.load(f)
            processed_files = set(checkpoint_data.get('processed_files', []))
        print(f"📋 Resuming from checkpoint: {len(processed_files)} files already processed")
    
    # Filter out already processed files
    remaining_files = [f for f in file_paths if f not in processed_files]
    print(f"🔄 {len(remaining_files)} files remaining to process")
    
    if not remaining_files:
        print("✅ All files already processed!")
        return load_existing_results()
    
    # Process remaining files
    all_results = load_existing_results() if processed_files else []
    
    for i, file_path in enumerate(remaining_files):
        print(f"Processing {i+1}/{len(remaining_files)}: {os.path.basename(file_path)}")
        
        result = processor.process_document(file_path)
        all_results.append(result)
        processed_files.add(file_path)
        
        # Save checkpoint periodically
        if (i + 1) % checkpoint_interval == 0:
            save_checkpoint(processed_files, all_results, checkpoint_path)
            print(f"📋 Checkpoint saved at {i+1} files")
    
    # Final save
    save_checkpoint(processed_files, all_results, checkpoint_path)
    
    # Convert to DataFrame and return
    df_results = create_results_dataframe(all_results)
    return df_results

def save_checkpoint(processed_files: set, results: list, checkpoint_path: str):
    """Save processing checkpoint"""
    
    checkpoint_data = {
        'processed_files': list(processed_files),
        'total_processed': len(processed_files),
        'last_updated': time.time()
    }
    
    os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
    
    with open(checkpoint_path, 'w') as f:
        json.dump(checkpoint_data, f, indent=2)
    
    # Save results
    results_path = "./results/interim_results.json"
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=2)

def load_existing_results() -> list:
    """Load existing results from checkpoint"""
    
    results_path = "./results/interim_results.json"
    
    if os.path.exists(results_path):
        with open(results_path, 'r') as f:
            return json.load(f)
    
    return []

# Example usage (uncomment to use):
# df_large = process_large_dataset("./your_large_dataset")

# %% [markdown]
# ## 10. Custom Field Patterns

# %%
def add_custom_extraction_patterns():
    """Add custom extraction patterns for specific contract types"""
    
    # Example: Department of Defense specific patterns
    dod_patterns = {
        'contract_number': [
            r'Contract(?:\s+No\.?|\s+Number)[:\s]*([A-Z0-9\-]+)',
            r'([A-Z]{2}\d{2}-\d{2}-[A-Z]-\d{4})',  # Standard DoD format
        ],
        'po_number': [
            r'P\.?O\.?\s*(?:Number|No\.?)[:\s]*([A-Z0-9\-]+)',
            r'Purchase Order[:\s]*([A-Z0-9\-]+)',
        ],
        'naics_code': [
            r'NAICS[:\s]*(\d{6})',
            r'Industry Code[:\s]*(\d{6})',
        ],
        'small_business': [
            r'Small Business[:\s]*(Yes|No|Y|N)',
            r'SB Status[:\s]*(Yes|No|Y|N)',
        ]
    }
    
    # Healthcare/FDA specific patterns
    fda_patterns = {
        'drug_code': [
            r'NDC[:\s]*(\d{5}-\d{4}-\d{2})',
            r'Drug Code[:\s]*([A-Z0-9\-]+)',
        ],
        'facility_number': [
            r'Facility[:\s]*(\d{7})',
            r'FEI[:\s]*(\d{10})',
        ]
    }
    
    print("📋 Custom extraction patterns available:")
    print("- Department of Defense (DoD)")
    print("- Food and Drug Administration (FDA)")
    print("\nTo use custom patterns, modify the parse_contract_fields method")
    
    return dod_patterns, fda_patterns

# Load custom patterns
dod_patterns, fda_patterns = add_custom_extraction_patterns()

# %% [markdown]
# ## 11. Quality Control and Validation

# %%
def validate_extraction_quality(df: pd.DataFrame, sample_size: int = 10):
    """Validate extraction quality on a sample of results"""
    
    print("🔍 QUALITY VALIDATION")
    print("=" * 30)
    
    if df.empty:
        print("No results to validate")
        return
    
    successful_df = df[df['status'] == 'success']
    
    if len(successful_df) == 0:
        print("No successful extractions to validate")
        return
    
    # Sample for validation
    sample_df = successful_df.sample(min(sample_size, len(successful_df)))
    
    print(f"Validating {len(sample_df)} samples...")
    
    # Quality metrics
    quality_metrics = {
        'files_with_agency': 0,
        'files_with_amount': 0,
        'files_with_vendor': 0,
        'files_with_dates': 0,
        'avg_fields_extracted': 0,
        'files_with_valid_amounts': 0
    }
    
    total_fields = 0
    
    for _, row in sample_df.iterrows():
        # Check for key fields
        if row.get('vendor_name', '').strip():
            quality_metrics['files_with_agency'] += 1
        
        if row.get('total_amount_this_action', '').strip() or row.get('new_contract_total', '').strip():
            quality_metrics['files_with_amount'] += 1
            
            # Validate amount format
            amount = row.get('total_amount_this_action', '') or row.get('new_contract_total', '')
            if re.match(r'^[\d,]+\.?\d*
    
    quality_metrics['avg_fields_extracted'] = total_fields / len(sample_df)
    
    # Display results
    print("\n📊 Quality Metrics:")
    for metric, value in quality_metrics.items():
        if metric == 'avg_fields_extracted':
            print(f"  {metric}: {value:.2f}")
        else:
            percentage = (value / len(sample_df)) * 100
            print(f"  {metric}: {value}/{len(sample_df)} ({percentage:.1f}%)")
    
    # Identify potential issues
    print("\n⚠️  Potential Issues:")
    if quality_metrics['files_with_agency'] < len(sample_df) * 0.8:
        print("  - Low agency name extraction rate")
    if quality_metrics['files_with_amount'] < len(sample_df) * 0.7:
        print("  - Low contract amount extraction rate")
    if quality_metrics['files_with_valid_amounts'] < quality_metrics['files_with_amount'] * 0.9:
        print("  - Amount format validation issues")
    
    return quality_metrics

# Validate sample quality
quality_metrics = validate_extraction_quality(df_results)

# %% [markdown]
# ## 12. Summary and Next Steps

# %%
def display_processing_summary():
    """Display final processing summary and recommendations"""
    
    print("🎉 CONTRACT PROCESSING PIPELINE COMPLETE")
    print("=" * 50)
    
    print("\n📋 What This Notebook Provides:")
    print("  ✓ Zero-shot contract field extraction")
    print("  ✓ Batch processing capabilities")
    print("  ✓ CPU-optimized processing")
    print("  ✓ Progress tracking and checkpointing")
    print("  ✓ Results analysis and visualization")
    print("  ✓ Multiple export formats (CSV, JSON, Excel)")
    print("  ✓ Quality validation tools")
    
    print("\n🚀 Next Steps for Your Research:")
    
    print("\n1. 📁 PREPARE YOUR DATA:")
    print("   - Organize contract files in a single directory")
    print("   - Ensure files are in supported formats (PDF, PNG, JPG)")
    print("   - Consider file naming convention for better organization")
    
    print("\n2. ⚙️  CONFIGURE PROCESSING:")
    print("   - Adjust batch_size based on your CPU memory")
    print("   - Modify extraction patterns for your specific contracts")
    print("   - Set up checkpoint directory for large datasets")
    
    print("\n3. 🔄 RUN PROCESSING:")
    print("   - Test on small sample first")
    print("   - Use large dataset processing for 200K documents")
    print("   - Monitor progress and handle any failures")
    
    print("\n4. 📊 ANALYZE RESULTS:")
    print("   - Validate extraction quality on samples")
    print("   - Export results in preferred format")
    print("   - Use visualizations for data exploration")
    
    print("\n5. 🔬 ACADEMIC ANALYSIS:")
    print("   - Clean and standardize extracted data")
    print("   - Perform statistical analysis")
    print("   - Document methodology for reproducibility")
    
    print("\n💡 Performance Expectations:")
    print("   - Speed: ~20-30 seconds per document on CPU")
    print("   - Accuracy: ~75-80% for zero-shot extraction")
    print("   - Scale: Can handle 200K+ documents")
    print("   - Timeline: 2-5 days for full dataset processing")
    
    print("\n📞 Support:")
    print("   - Test with sample documents first")
    print("   - Check logs for processing errors")
    print("   - Adjust extraction patterns as needed")
    print("   - Use quality validation to assess results")

# Display final summary
display_processing_summary()

# %%
print("\n✅ NOTEBOOK SETUP COMPLETE!")
print("\nTo process your contracts:")
print("1. Update INPUT_DIRECTORY in Section 6")
print("2. Uncomment the processing line")
print("3. Run the processing cell")
print("4. Use the analysis and export functions")

print(f"\n📊 Current Configuration:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

print("\n🎯 Ready to process government contracts at scale!")

# %% [markdown]
# ---
# 
# ## Additional Notes
# 
# **For Academic Use:**
# - This notebook uses models that are free for academic research
# - LayoutLMv3 can be substituted if you need higher accuracy
# - All processing is done locally - no data leaves your environment
# 
# **Performance Tips:**
# - Start with a small sample to test extraction patterns
# - Adjust batch_size based on available CPU memory
# - Use checkpointing for very large datasets
# - Consider preprocessing PDFs to images for better consistency
# 
# **Customization:**
# - Modify regex patterns in `parse_contract_fields()` for your specific contract formats
# - Add new field types by extending the extraction patterns
# - Implement custom validation rules for your use case
# 
# **Troubleshooting:**
# - Check model download in cache directory
# - Verify PDF processing dependencies (poppler-utils)
# - Monitor memory usage during batch processing
# - Use quality validation to identify extraction issues, amount.replace('
    
    quality_metrics['avg_fields_extracted'] = total_fields / len(sample_df)
    
    # Display results
    print("\n📊 Quality Metrics:")
    for metric, value in quality_metrics.items():
        if metric == 'avg_fields_extracted':
            print(f"  {metric}: {value:.2f}")
        else:
            percentage = (value / len(sample_df)) * 100
            print(f"  {metric}: {value}/{len(sample_df)} ({percentage:.1f}%)")
    
    # Identify potential issues
    print("\n⚠️  Potential Issues:")
    if quality_metrics['files_with_agency'] < len(sample_df) * 0.8:
        print("  - Low agency name extraction rate")
    if quality_metrics['files_with_amount'] < len(sample_df) * 0.7:
        print("  - Low contract amount extraction rate")
    if quality_metrics['files_with_valid_amounts'] < quality_metrics['files_with_amount'] * 0.9:
        print("  - Amount format validation issues")
    
    return quality_metrics

# Validate sample quality
quality_metrics = validate_extraction_quality(df_results)

# %% [markdown]
# ## 12. Summary and Next Steps

# %%
def display_processing_summary():
    """Display final processing summary and recommendations"""
    
    print("🎉 CONTRACT PROCESSING PIPELINE COMPLETE")
    print("=" * 50)
    
    print("\n📋 What This Notebook Provides:")
    print("  ✓ Zero-shot contract field extraction")
    print("  ✓ Batch processing capabilities")
    print("  ✓ CPU-optimized processing")
    print("  ✓ Progress tracking and checkpointing")
    print("  ✓ Results analysis and visualization")
    print("  ✓ Multiple export formats (CSV, JSON, Excel)")
    print("  ✓ Quality validation tools")
    
    print("\n🚀 Next Steps for Your Research:")
    
    print("\n1. 📁 PREPARE YOUR DATA:")
    print("   - Organize contract files in a single directory")
    print("   - Ensure files are in supported formats (PDF, PNG, JPG)")
    print("   - Consider file naming convention for better organization")
    
    print("\n2. ⚙️  CONFIGURE PROCESSING:")
    print("   - Adjust batch_size based on your CPU memory")
    print("   - Modify extraction patterns for your specific contracts")
    print("   - Set up checkpoint directory for large datasets")
    
    print("\n3. 🔄 RUN PROCESSING:")
    print("   - Test on small sample first")
    print("   - Use large dataset processing for 200K documents")
    print("   - Monitor progress and handle any failures")
    
    print("\n4. 📊 ANALYZE RESULTS:")
    print("   - Validate extraction quality on samples")
    print("   - Export results in preferred format")
    print("   - Use visualizations for data exploration")
    
    print("\n5. 🔬 ACADEMIC ANALYSIS:")
    print("   - Clean and standardize extracted data")
    print("   - Perform statistical analysis")
    print("   - Document methodology for reproducibility")
    
    print("\n💡 Performance Expectations:")
    print("   - Speed: ~20-30 seconds per document on CPU")
    print("   - Accuracy: ~75-80% for zero-shot extraction")
    print("   - Scale: Can handle 200K+ documents")
    print("   - Timeline: 2-5 days for full dataset processing")
    
    print("\n📞 Support:")
    print("   - Test with sample documents first")
    print("   - Check logs for processing errors")
    print("   - Adjust extraction patterns as needed")
    print("   - Use quality validation to assess results")

# Display final summary
display_processing_summary()

# %%
print("\n✅ NOTEBOOK SETUP COMPLETE!")
print("\nTo process your contracts:")
print("1. Update INPUT_DIRECTORY in Section 6")
print("2. Uncomment the processing line")
print("3. Run the processing cell")
print("4. Use the analysis and export functions")

print(f"\n📊 Current Configuration:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

print("\n🎯 Ready to process government contracts at scale!")

# %% [markdown]
# ---
# 
# ## Additional Notes
# 
# **For Academic Use:**
# - This notebook uses models that are free for academic research
# - LayoutLMv3 can be substituted if you need higher accuracy
# - All processing is done locally - no data leaves your environment
# 
# **Performance Tips:**
# - Start with a small sample to test extraction patterns
# - Adjust batch_size based on available CPU memory
# - Use checkpointing for very large datasets
# - Consider preprocessing PDFs to images for better consistency
# 
# **Customization:**
# - Modify regex patterns in `parse_contract_fields()` for your specific contract formats
# - Add new field types by extending the extraction patterns
# - Implement custom validation rules for your use case
# 
# **Troubleshooting:**
# - Check model download in cache directory
# - Verify PDF processing dependencies (poppler-utils)
# - Monitor memory usage during batch processing
# - Use quality validation to identify extraction issues, '').strip()):
                quality_metrics['files_with_valid_amounts'] += 1
        
        if row.get('vendor_name', '').strip():
            quality_metrics['files_with_vendor'] += 1
        
        if row.get('from_date', '').strip() and row.get('to_date', '').strip():
            quality_metrics['files_with_dates'] += 1
        
        # Count total fields extracted
        field_count = sum(1 for field in ['eds_number', 'vendor_name', 'total_amount_this_action', 
                                        'from_date', 'to_date', 'account_number']
                         if row.get(field, '').strip())
        total_fields += field_count
    
    quality_metrics['avg_fields_extracted'] = total_fields / len(sample_df)
    
    # Display results
    print("\n📊 Quality Metrics:")
    for metric, value in quality_metrics.items():
        if metric == 'avg_fields_extracted':
            print(f"  {metric}: {value:.2f}")
        else:
            percentage = (value / len(sample_df)) * 100
            print(f"  {metric}: {value}/{len(sample_df)} ({percentage:.1f}%)")
    
    # Identify potential issues
    print("\n⚠️  Potential Issues:")
    if quality_metrics['files_with_agency'] < len(sample_df) * 0.8:
        print("  - Low agency name extraction rate")
    if quality_metrics['files_with_amount'] < len(sample_df) * 0.7:
        print("  - Low contract amount extraction rate")
    if quality_metrics['files_with_valid_amounts'] < quality_metrics['files_with_amount'] * 0.9:
        print("  - Amount format validation issues")
    
    return quality_metrics

# Validate sample quality
quality_metrics = validate_extraction_quality(df_results)

# %% [markdown]
# ## 12. Summary and Next Steps

# %%
def display_processing_summary():
    """Display final processing summary and recommendations"""
    
    print("🎉 CONTRACT PROCESSING PIPELINE COMPLETE")
    print("=" * 50)
    
    print("\n📋 What This Notebook Provides:")
    print("  ✓ Zero-shot contract field extraction")
    print("  ✓ Batch processing capabilities")
    print("  ✓ CPU-optimized processing")
    print("  ✓ Progress tracking and checkpointing")
    print("  ✓ Results analysis and visualization")
    print("  ✓ Multiple export formats (CSV, JSON, Excel)")
    print("  ✓ Quality validation tools")
    
    print("\n🚀 Next Steps for Your Research:")
    
    print("\n1. 📁 PREPARE YOUR DATA:")
    print("   - Organize contract files in a single directory")
    print("   - Ensure files are in supported formats (PDF, PNG, JPG)")
    print("   - Consider file naming convention for better organization")
    
    print("\n2. ⚙️  CONFIGURE PROCESSING:")
    print("   - Adjust batch_size based on your CPU memory")
    print("   - Modify extraction patterns for your specific contracts")
    print("   - Set up checkpoint directory for large datasets")
    
    print("\n3. 🔄 RUN PROCESSING:")
    print("   - Test on small sample first")
    print("   - Use large dataset processing for 200K documents")
    print("   - Monitor progress and handle any failures")
    
    print("\n4. 📊 ANALYZE RESULTS:")
    print("   - Validate extraction quality on samples")
    print("   - Export results in preferred format")
    print("   - Use visualizations for data exploration")
    
    print("\n5. 🔬 ACADEMIC ANALYSIS:")
    print("   - Clean and standardize extracted data")
    print("   - Perform statistical analysis")
    print("   - Document methodology for reproducibility")
    
    print("\n💡 Performance Expectations:")
    print("   - Speed: ~20-30 seconds per document on CPU")
    print("   - Accuracy: ~75-80% for zero-shot extraction")
    print("   - Scale: Can handle 200K+ documents")
    print("   - Timeline: 2-5 days for full dataset processing")
    
    print("\n📞 Support:")
    print("   - Test with sample documents first")
    print("   - Check logs for processing errors")
    print("   - Adjust extraction patterns as needed")
    print("   - Use quality validation to assess results")

# Display final summary
display_processing_summary()

# %%
print("\n✅ NOTEBOOK SETUP COMPLETE!")
print("\nTo process your contracts:")
print("1. Update INPUT_DIRECTORY in Section 6")
print("2. Uncomment the processing line")
print("3. Run the processing cell")
print("4. Use the analysis and export functions")

print(f"\n📊 Current Configuration:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

print("\n🎯 Ready to process government contracts at scale!")

# %% [markdown]
# ---
# 
# ## Additional Notes
# 
# **For Academic Use:**
# - This notebook uses models that are free for academic research
# - LayoutLMv3 can be substituted if you need higher accuracy
# - All processing is done locally - no data leaves your environment
# 
# **Performance Tips:**
# - Start with a small sample to test extraction patterns
# - Adjust batch_size based on available CPU memory
# - Use checkpointing for very large datasets
# - Consider preprocessing PDFs to images for better consistency
# 
# **Customization:**
# - Modify regex patterns in `parse_contract_fields()` for your specific contract formats
# - Add new field types by extending the extraction patterns
# - Implement custom validation rules for your use case
# 
# **Troubleshooting:**
# - Check model download in cache directory
# - Verify PDF processing dependencies (poppler-utils)
# - Monitor memory usage during batch processing
# - Use quality validation to identify extraction issues